<a href="https://colab.research.google.com/github/Kostogot/Two/blob/main/Binan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
 import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None) 
B = pd.read_excel('/binan.xlsx', 1)
#B

In [ ]:
 
A=[] 
for n in range(0,4):
  if B['operation'][n] == 'buy':
    B['%'][n] = -B['fee'][n] 
    A.append([B['volume '][n],n])
B
A

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


[[0.0339041, 0], [0.163, 1]]

In [22]:
#B['volume ']

In [ ]:
A=[[4,2], [5,0]]
A.append([3,4])
A.sort()
A.reverse()
A

[[5, 0], [4, 2], [3, 4]]